In [ ]:
# Example 19: FM radio - frequency spectrum 
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import asyncio
from rtlsdr import RtlSdr
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import welch
from collections import deque
import linecache
import sys

# -------------- global variables -------------------        

MEAS_RUNNING = True      # when switched to False all processes are stopped

# configuration SDR
# for radio
BIAS_TEE = False        # set True for GPS antenna
SAMPLE_RATE = 2.048e6   
FREQ_CORRECTION = -1    # in ppm
CENTER_FREQ = 106.3e6   # FM in range 87.5e6 - 108e6
GAIN = 6                #'auto' or 0..49.6; tuner GAIN in dB

AVG = 128               # average for FFT
NSMP = AVG*2048         # number of samples per cycle      

MEAS_TIME = 300.0       # in seconds

DATA_FILE = ''          # if not empty, file is used instead of
                        # real-time data

# ------- Exception handling  -------------------

EXC = []

def printException():
    global EXC
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    EXC.append(('EXCEPTION IN ({}, LINE {} "{}"): {}'.\
                format(filename, lineno, line.strip(), exc_obj)))        
    

# -------- keyboard events ---------------

def onKeypress(event):
    global STATUS_MSG
    global MEAS_RUNNING
    sys.stdout.flush()
    if event.key in ['q','Q']:
        MEAS_RUNNING = False
        STATUS_MSG = 'q pressed'

# ------- data buffer -----------------

MAXBUFSIZE = 16     
BUFFER = deque([],maxlen=MAXBUFSIZE)
NBUF = 0            
BUFSKIP = 0


def pushToBuffer(data):
    global BUFFER
    global NBUF
    global BUFSKIP

    if NBUF >= MAXBUFSIZE:
        BUFFER.clear()
        NBUF = 0
        BUFSKIP += MAXBUFSIZE
        
    BUFFER.append(data)
    NBUF += 1
    
    
def pullFromBuffer():
    global BUFFER
    global NBUF
    global BUFSKIP

    try:
        data = BUFFER.popleft()
        NBUF -= 1
        skip = BUFSKIP
        BUFSKIP = 0
    except IndexError:
        data = []
        skip = 0
    
    return data,skip

# ------- calc power spectrum or power spectral density  ---------

def calcWelch(y,NAvg,SAMPLE_RATE,scaling):
    # scaling: 'density' or 'spectrum'    
    nfft = len(y)//NAvg
    # rms depend on window; default is 'hann'; also possible e.g. 'boxcar'
    _,psd = welch(y,SAMPLE_RATE,nperseg=nfft,scaling=scaling)  
    # rms value of time signal (Parseval theorem)
    rms = np.sum(psd) if scaling == 'spectrum' else None       
    # shift 0 to center   
    psd = np.fft.fftshift(psd)                                
    
    return psd,rms


def calcBartlett(y,NAvg,SAMPLE_RATE,scaling): 
    # scaling: 'density' or 'spectrum'    
    psd,rms = None,None
    m = len(y)//NAvg
    for i in range(NAvg):
        zf = np.abs(fft(y[i*m:(i+1)*m]))**2
        if i==0:
            mf = zf
        else:
            mf += zf
    mf /= NAvg
    if scaling == 'density':
        psd = mf/(m*SAMPLE_RATE)
    elif scaling == 'spectrum':
        psd = mf/m**2
        # rms value of time signal (Parseval theorem)        
        rms = np.sum(psd)      
    # shift 0 to center
    psd = np.fft.fftshift(psd) 
        
    return psd,rms


# ------- plot data -------------------

def plot(y):
    line1.set_ydata(y)       
    fig.canvas.draw_idle()
    
# ------- process data ------------------

async def processData():   
    global MEAS_RUNNING
    global rms1,rms2
    try:
        while MEAS_RUNNING:
            if NBUF >= 1:
                y,skip = pullFromBuffer()
                rms1 = np.mean(abs(y)**2)
                # scaling is 'spectrum' or 'density'                
                #psd,rms2 = calcBartlett(y,AVG,SAMPLE_RATE,'spectrum')
                psd,rms2 = calcWelch(y,AVG,SAMPLE_RATE,'spectrum')
                psd = 10*np.log10(psd)            # calc dB from power
                plot(psd)
                if skip > 0:
                    raise ValueError('buffer overload')
            await asyncio.sleep(0)
    except:
        printException()
    finally:
        MEAS_RUNNING = False

# ----------- Streaming ----------------        

async def streamData():
    global MEAS_RUNNING
    start_time = time.time()
    end_time = start_time+MEAS_TIME
    try:
        with open(DATA_FILE,'rb') as f1:
            while MEAS_RUNNING:
                if NBUF < NSMP:
                    samples = np.fromfile(f1,dtype=np.complex64,count=NSMP)
                    if len(samples)>0:
                        pushToBuffer(samples)
                    else:
                        MEAS_RUNNING = False
                if time.time()>end_time:                
                    MEAS_RUNNING = False                                            
                await asyncio.sleep(0)
    except BaseException as err:
        printException()
    finally:
        MEAS_RUNNING = False        
        await task2
        

async def streamLive():
    global MEAS_RUNNING
    
    sdr = RtlSdr()
    sdr.set_bias_tee(BIAS_TEE)
    sdr.sample_rate = SAMPLE_RATE
    sdr.freq_correction = FREQ_CORRECTION    
    sdr.center_freq = CENTER_FREQ   
    sdr.gain = GAIN 
        
    start_time = time.time()
    end_time = start_time + MEAS_TIME
    try:
        async for samples in sdr.stream(num_samples_or_bytes=NSMP, 
                                        format='samples'):
            pushToBuffer(samples)
            if time.time()>end_time:                
                MEAS_RUNNING = False                    
            if not MEAS_RUNNING:
                await sdr.stop()
                sdr.close()
    except:    
        printException()
    finally:
        MEAS_RUNNING = False

# --------------- prepare plot ------------------        


# values for frequency axis
xf = fftfreq(NSMP//AVG,1/SAMPLE_RATE)/1.0e6     # in MHz
xf = np.fft.fftshift(xf)                         # center 0
xf += CENTER_FREQ/1.0e6

yf = np.zeros(NSMP//AVG)                        # for inital plot
        
fig,ax = plt.subplots(figsize=(6,4))
fig.canvas.header_visible = False
fig.canvas.mpl_connect('key_press_event', onKeypress)   

# plot of frequency spectrum
line1, = ax.plot(xf,yf,lw=.5)
ax.set_ylim(-70,0)     # radio
#ax.set_ylim(-60,-30)   # GPS 
ax.set_xlabel('Frequency (MHz)')
# power spectral density (not calibrated, dB full scale)
ax.set_ylabel('Power (dBfs)')        
plt.tight_layout()

print('Click on graph and press q to exit!')

plt.show()

# ------ Main ---------------------------

# start async tasks
loop = asyncio.get_event_loop()
task2 = loop.create_task(processData())
if DATA_FILE =='':
    task1 = loop.create_task(streamLive())
else:
    task1 = loop.create_task(streamData())    
    
